In [1]:
import json
from tqdm import tqdm_notebook
import gzip
import pandas as pd
import sys
import json
from typing import Dict, List
import os
import numpy as np

In [2]:
from os import walk

f_zips = []
for (dirpath, dirnames, filenames) in walk('../../gorc/'):
    f_zips.extend(filenames)
    break

In [3]:
len(f_zips),f_zips

(10002,
 ['0.jsonl.gz',
  '1.jsonl.gz',
  '10.jsonl.gz',
  '100.jsonl.gz',
  '1000.jsonl.gz',
  '1001.jsonl.gz',
  '1002.jsonl.gz',
  '1003.jsonl.gz',
  '1004.jsonl.gz',
  '1005.jsonl.gz',
  '1006.jsonl.gz',
  '1007.jsonl.gz',
  '1008.jsonl.gz',
  '1009.jsonl.gz',
  '101.jsonl.gz',
  '1010.jsonl.gz',
  '1011.jsonl.gz',
  '1012.jsonl.gz',
  '1013.jsonl.gz',
  '1014.jsonl.gz',
  '1015.jsonl.gz',
  '1016.jsonl.gz',
  '1017.jsonl.gz',
  '1018.jsonl.gz',
  '1019.jsonl.gz',
  '102.jsonl.gz',
  '1020.jsonl.gz',
  '1021.jsonl.gz',
  '1022.jsonl.gz',
  '1023.jsonl.gz',
  '1024.jsonl.gz',
  '1025.jsonl.gz',
  '1026.jsonl.gz',
  '1027.jsonl.gz',
  '1028.jsonl.gz',
  '1029.jsonl.gz',
  '103.jsonl.gz',
  '1030.jsonl.gz',
  '1031.jsonl.gz',
  '1032.jsonl.gz',
  '1033.jsonl.gz',
  '1034.jsonl.gz',
  '1035.jsonl.gz',
  '1036.jsonl.gz',
  '1037.jsonl.gz',
  '1038.jsonl.gz',
  '1039.jsonl.gz',
  '104.jsonl.gz',
  '1040.jsonl.gz',
  '1041.jsonl.gz',
  '1042.jsonl.gz',
  '1043.jsonl.gz',
  '1044.jsonl.gz'

In [4]:
[file for file in f_zips if '.gz' not in file]

['s2orc-master.zip']

#### Загрузим все  acl статьи

In [5]:
with open("acl_only_json_list_10000.json", "r") as read_file:
    acl_only_articles = json.load(read_file)
print(len(acl_only_articles))
read_file.close()

41660


In [6]:
len(acl_only_articles)

41660

#### Выберем статьи из acl
чтобы по ним выкачать статьи из reference c аннотацией, которые больше всего подходят для формирования обучающей выборки сценариев

In [7]:
len([article['paper_id'] for article in acl_only_articles if article['grobid_parse']['body_text'] or (article['latex_parse'] and article['latex_parse']['body_text'])])

41439

##### Удалим статьи,у которых нет body_text

In [8]:
acl_ids_not_body_text = [article['paper_id'] for article in acl_only_articles if not article['grobid_parse']['body_text'] or not article['grobid_parse']['bib_entries']]

In [9]:
len(acl_ids_not_body_text)

1897

In [10]:
def delete_items_from_papers(acl_ids_not_body_text,acl_only_articles):
    del_items = []
    del_num_items = []
    for num_artic, article in enumerate(acl_only_articles):
        if article['paper_id'] in acl_ids_not_body_text:
            del_items.append(article['paper_id'])
            del_num_items.append(num_artic)
            
    del_num_items = np.array(del_num_items)
    acl_only_articles = np.array(acl_only_articles)
    acl_only_articles = np.delete(acl_only_articles,del_num_items)
    return acl_only_articles

In [11]:
len(acl_only_articles)

41660

In [12]:
acl_only_articles = delete_items_from_papers(acl_ids_not_body_text,acl_only_articles)

In [13]:
len(acl_only_articles)

39763

##### удалим дублированные статьи по acl_id

In [14]:
acl_paper_ids = [article['metadata']['acl_id'] for article in acl_only_articles if article['metadata']['acl_id']]
len(acl_paper_ids)

39763

In [15]:
doubled_acl_id_papers = []
for ind,cnt_of_acl_id in zip(pd.Series(acl_paper_ids).value_counts().index,pd.Series(acl_paper_ids).value_counts()):
    if cnt_of_acl_id >=2:
        doubled_acl_id_papers.append(ind)

In [16]:
len(doubled_acl_id_papers)

284

In [17]:
del_items = []
del_num_items = []
for num_artic, article in enumerate(acl_only_articles):
    if article['metadata']['acl_id'] in doubled_acl_id_papers:
        del_items.append(article['paper_id'])
        del_num_items.append(num_artic)

In [18]:
len(del_items)

572

In [19]:
acl_only_articles = np.delete(acl_only_articles,del_num_items)

In [20]:
del_items = []
del_num_items = []
for num_artic, article in enumerate(acl_only_articles):
    if article['metadata']['acl_id'] in doubled_acl_id_papers:
        del_items.append(article['paper_id'])
        del_num_items.append(num_artic)

In [21]:
len(del_items)

0

In [22]:
acl_paper_ids = [article['metadata']['acl_id'] for article in acl_only_articles if article['metadata']['acl_id']]
len(acl_paper_ids)

39191

#### Выделим обзорные части статей

In [23]:
overview_papers = dict()
for num_artic,article in enumerate(acl_only_articles):
    # проверяем что у статьи есть grobid_parse и latex_parse и естб текст
    if (article['grobid_parse'] and article['grobid_parse']['body_text']) or (article['latex_parse'] and article['latex_parse']['body_text']):
        # задаем шаблон отражения статьи в укороченном формате (чтобы занимать меньше памяти)
        overview_papers[article['paper_id']] = { 'paper_id':article['paper_id'],   'metadata':article['metadata'],
                                                 's2_pdf_hash':article['s2_pdf_hash'], 'grobid_parse':None,'latex_parse':None}
        
        grobid_parse_overview = None
        # если у статьи есть article['grobid_parse']['body_text']
        if article['grobid_parse'] and article['grobid_parse']['body_text']:
            grobid_parse_overview = dict()
            # проходим по каждому абзацу текста
            for num_sec,sections in enumerate(article['grobid_parse']['body_text']):
                grobid_parse_overview[num_sec] = sections
            
            # отсортируем по количеству цитат абзацы
            
            grobid_parse_overview = {k: v for k, v in sorted(grobid_parse_overview.items(), key=lambda item: len(item[1]['cite_spans']), reverse=True)}
            
#             # найдем 1 , пока без 2 максимума по обзорной части
            max_cite_span_sum = 0
            max_grobid_parse_overview = dict()
            for k,v in grobid_parse_overview.items():
                if max_cite_span_sum < len(v['cite_spans']):
                    max_cite_span_sum = len(v['cite_spans'])
                    max_grobid_parse_overview[k] = v
                    
#                 # записываем 2 максимум, если количество ссылок в егочасти больше половины от максимального 
#                 elif (max_cite_span_sum>7) and len(v['cite_spans'])>max_cite_span_sum//2:
#                     max_grobid_parse_overview[k] = v
                    
            
            grobid_parse_overview = max_grobid_parse_overview
            
        latex_parse_overview = None
        # если у статьи есть article['latex_parse']['body_text']
        if article['latex_parse'] and article['latex_parse']['body_text']:
            latex_parse_overview = dict()
            # проходим по каждому абзацу текста
            # в latex_parse 
            for sections in article['latex_parse']['body_text']:
                if sections['section'] in latex_parse_overview:
                    if latex_parse_overview[sections['section']] == sections:
                        continue
                    else:
                        latex_parse_overview[sections['section']]['text'].append(sections['text'])
                        latex_parse_overview[sections['section']]['cite_spans'].append(sections['cite_spans'])
                        latex_parse_overview[sections['section']]['cite_span_lens'].append(len(sections['cite_spans']))
                        latex_parse_overview[sections['section']]['section'].append(sections['section'])
                else:
                    latex_parse_overview[sections['section']] = {'text':[sections['text']],   'cite_spans':[sections['cite_spans']],
                                                                  'cite_span_lens':[len(sections['cite_spans'])],
                                                                  'section':[sections['section']]}
            latex_parse_overview = {k: v for k, v in sorted(latex_parse_overview.items(), key=lambda item: item[1]['cite_span_lens'], reverse=True)}
        
        
            max_cite_span_sum = 0
            max_latex_parse_overview = dict()
            for k,v in latex_parse_overview.items():
                if max_cite_span_sum < sum(v['cite_span_lens']):
                    max_cite_span_sum = sum(v['cite_span_lens'])
                    max_latex_parse_overview[k] = v
#                 elif (max_cite_span_sum>0) and sum(v['cite_span_lens'])>max_cite_span_sum//2:
#                     max_latex_parse_overview[k] = v
            
            latex_parse_overview = max_latex_parse_overview
        

        if grobid_parse_overview:
            overview_papers[article['paper_id']]['grobid_parse'] = {'abstract':None,
                                                        'overview_text':grobid_parse_overview,  
                                                        'bib_entries':None}
            if article['grobid_parse']['abstract']:
                overview_papers[article['paper_id']]['grobid_parse']['abstract'] = article['grobid_parse']['abstract']
            if article['grobid_parse']['bib_entries']:
                overview_papers[article['paper_id']]['grobid_parse']['bib_entries'] = article['grobid_parse']['bib_entries']
            
        if latex_parse_overview:            
            overview_papers[article['paper_id']]['latex_parse'] = {'abstract':None,
                                                                    'overview_text':latex_parse_overview,  
                                                                    'bib_entries':None}
            if article['latex_parse']['abstract']:
                overview_papers[article['paper_id']]['latex_parse']['abstract'] = article['latex_parse']['abstract']
            if article['latex_parse']['bib_entries']:
                overview_papers[article['paper_id']]['latex_parse']['bib_entries'] = article['latex_parse']['bib_entries']

In [24]:
len(overview_papers),len(acl_only_articles)

(39191, 39191)

In [25]:
list(overview_papers.keys())[:10]

['10164018',
 '14472576',
 '17302615',
 '3243536',
 '3248240',
 '2223737',
 '488',
 '14323173',
 '15251605',
 '8260435']

In [26]:
overview_papers['10164018']

{'paper_id': '10164018',
 'metadata': {'title': 'Reader-Aware Multi-Document Summarization: An Enhanced Model and The First Dataset',
  'authors': [{'first': 'Piji', 'middle': [], 'last': 'Li', 'suffix': ''},
   {'first': 'Lidong', 'middle': [], 'last': 'Bing', 'suffix': ''},
   {'first': 'Wai', 'middle': [], 'last': 'Lam', 'suffix': ''}],
  'abstract': 'We investigate the problem of reader-aware multi-document summarization (RA-MDS) and introduce a new dataset for this problem. To tackle RA-MDS, we extend a variational auto-encodes (VAEs) based MDS framework by jointly considering news documents and reader comments. To conduct evaluation for summarization performance, we prepare a new dataset. We describe the methods for data collection, aspect annotation, and summary writing as well as scrutinizing by experts. Experimental results show that reader comments can improve the summarization performance, which also demonstrates the usefulness of the proposed dataset. The annotated dataset 

#### Выделим количество ссылок

In [27]:
article_bibs = {}
for num_artic,article in tqdm_notebook(enumerate(overview_papers)):
# for num_artic,article in enumerate(overview_papers):
#     if num_artic >10:
#         break
#     print(num_artic,article)
    article = overview_papers[article]
    # проверка наличия определленных значений статьи
    # если нет в grobid_parse bib_entries, то нет смысла рассматривать статью, тк отсутствуют cite_spans
    if not ((article['grobid_parse'] and article['grobid_parse']['bib_entries']) or (article['latex_parse'] and article['latex_parse']['bib_entries'])):
#         article_bibs[article['paper_id']] = 'None'
#         print(num_artic,article['paper_id'])
        continue
        
    bib_entries_grobid = None
    if article['grobid_parse'] and article['grobid_parse']['bib_entries'] and article['grobid_parse']['overview_text']:
        bib_entries_grobid = dict()
        
        key_grobid = list(article['grobid_parse']['overview_text'].keys())[0]
        for cite_span in article['grobid_parse']['overview_text'][key_grobid]['cite_spans']:
            cite_ref = cite_span['ref_id']
            cited_paper_id = None
            if cite_ref in article['grobid_parse']['bib_entries']:
                cited_paper_id = article['grobid_parse']['bib_entries'][cite_ref]['links']
            if cited_paper_id:
                bib_entries_grobid[article['grobid_parse']['bib_entries'][cite_ref]['title']] = cited_paper_id
            elif cite_ref  in article['grobid_parse']['bib_entries']:
                bib_entries_grobid[article['grobid_parse']['bib_entries'][cite_ref]['title']] = cited_paper_id
        
        
        
#     bib_entries_latex = None
#     if article['latex_parse'] and article['latex_parse']['overview_text'] and article['latex_parse']['bib_entries']:
#         bib_entries_latex = dict()
        
#         key_tex = list(article['latex_parse']['overview_text'].keys())[0]
#         for cite_spans in article['latex_parse']['overview_text'][key_tex]['cite_spans']:
#             if len(cite_spans)>1:
#                 for cite_span in cite_spans:
#                     cite_ref = cite_span['ref_id']
#                     cited_article_id = None
#                     if cite_ref in article['latex_parse']['bib_entries']:
#                         cited_article_id = article['latex_parse']['bib_entries'][cite_ref]['links']
#                     if cited_article_id:
#                         bib_entries_latex[article['latex_parse']['bib_entries'][cite_ref]['title']] = cited_article_id
#                     elif cite_ref in article['latex_parse']['bib_entries']:
#                         bib_entries_latex[article['latex_parse']['bib_entries'][cite_ref]['title']] = cited_article_id
    
    
    all_bib_entries = dict()
#     if bib_entries_latex:
#         for k,v in bib_entries_latex.items():
#             if k == '':
#                 continue
                
#             if v:
#                 all_bib_entries[k] = [v]
#             else:
#                 all_bib_entries[k] = []

    if bib_entries_grobid:
        for k,v in bib_entries_grobid.items():
            if k == '':
                continue
                
            if k in all_bib_entries:
                if len(all_bib_entries[k]) == 0:
                    if v:
                        all_bib_entries[k] = [v]
                    else:
                        all_bib_entries[k] = []
                else:
                    if all_bib_entries[k][0] == v:
                        continue
                    else:
                        if v:
                            all_bib_entries[k].append(v)
                        else:
                            continue
            else:
                if v:
                    all_bib_entries[k] = [v]
                else:
                    all_bib_entries[k] = []
    article_bibs[article['paper_id']] = all_bib_entries

In [28]:
len(article_bibs)

38824

In [29]:
article_bibs_keys = {val:1 for val in article_bibs}

In [30]:
acl_ids_not_bibs = [article['paper_id'] for article in acl_only_articles if article['paper_id'] not in article_bibs_keys]

In [31]:
len(acl_ids_not_bibs),acl_ids_not_bibs[:5]

(367, ['65364185', '18414606', '8825607', '1294235', '18312930'])

In [32]:
acl_only_articles = delete_items_from_papers(acl_ids_not_bibs,acl_only_articles)

In [33]:
len(acl_only_articles)

38824

In [34]:
list(article_bibs_keys.keys())[:6]

['10164018', '14472576', '17302615', '3243536', '3248240', '2223737']

In [35]:
article_bibs['10164018']

{'Multi-document summarization by sentence extraction': ['8294822'],
 'Lexpagerank: Prestige in multi-document text summarization': ['10418456'],
 'Manifold-ranking based topic-focused multidocument summarization': ['532313'],
 'A survey of text summarization techniques': ['556431'],
 'Exploiting category-specific information for multidocument summarization': ['6317274'],
 'Salience estimation via variational auto-encoders for multi-document summarization': ['29562039'],
 "Comments-oriented document summarization: Understanding documents with readers' feedback": ['13723748'],
 'Social context summarization': ['704517'],
 'Abstractive multidocument summarization via phrase selection and merging': ['8377315'],
 'Autoencoding variational bayes': [],
 'Stochastic backpropagation and approximate inference in deep generative models': ['16895865']}

3 ячейки ниже просто чтобы работало 

In [36]:
with open('all_in_paper.txt', encoding="utf8") as fin:
    content = fin.readlines()
fin.close()

In [37]:
all_dict_in_ids = [x.strip() for x in content]
del content

In [38]:
dict_in_ids_all = {val:1 for val in all_dict_in_ids}
len(dict_in_ids_all)

5986243

In [39]:
index = []
num_all_bibs = []
num_all_bibs_with_links = []
num_all_bibs_w_l_in_dict_in = []
for num_artic,(key,val_dict) in enumerate(article_bibs.items()):
    index.append(key)
    num_bibs = 0
    num_bibs_with_links = 0
    num_bibs_w_l_in_dict_in = 0
    try:
        for k,val in val_dict.items():
            num_bibs +=1
            if len(val)>0:
                num_bibs_with_links+=1
            for link in val:
                if link in dict_in_ids_all:
                    num_bibs_w_l_in_dict_in+=1
                    break
    except:
        print(num_artic)
        print(key)
        break
    num_all_bibs.append(num_bibs)
    num_all_bibs_with_links.append(num_bibs_with_links)
    num_all_bibs_w_l_in_dict_in.append(num_bibs_w_l_in_dict_in)

In [40]:
d_overview = {'#bibs':num_all_bibs,'#bibs_with_links':num_all_bibs_with_links,'#bibs_with_links_in_dict':num_all_bibs_w_l_in_dict_in}
df_covering_overview =  pd.DataFrame(d_overview)
df_covering_overview.index = index

In [41]:
df_covering_overview['%_in_dict2bibs'] = df_covering_overview['#bibs_with_links_in_dict']/df_covering_overview['#bibs']
df_covering_overview['%_in_dict2bibs_w_l'] = df_covering_overview['#bibs_with_links_in_dict']/df_covering_overview['#bibs_with_links']
df_covering_overview['%bibs_with_links2bibs'] = df_covering_overview['#bibs_with_links']/df_covering_overview['#bibs']

In [42]:
df_covering_overview.head()

,#bibs,#bibs_with_links,#bibs_with_links_in_dict,%_in_dict2bibs,%_in_dict2bibs_w_l,%bibs_with_links2bibs
10164018,11,10,5,0.454545,0.500000,0.909091
14472576,18,18,16,0.888889,0.888889,1.000000
17302615,3,1,0,0.000000,0.000000,0.333333
3243536,2,1,1,0.500000,1.000000,0.500000
3248240,4,3,3,0.750000,1.000000,0.750000


In [43]:
df_covering_overview['%bibs_with_links2bibs'].mean()

0.7064105122470907

In [44]:
df_covering_overview[df_covering_overview['%bibs_with_links2bibs']>=0.9].shape

(10020, 6)

In [45]:
df_covering_overview[(df_covering_overview['%bibs_with_links2bibs']>=0.9) & (df_covering_overview['#bibs']>=5)].shape

(7259, 6)

In [46]:
df_covering_overview[df_covering_overview['%bibs_with_links2bibs']>=0.8].shape

(17750, 6)

In [47]:
df_covering_overview[(df_covering_overview['%bibs_with_links2bibs']>=0.8) & (df_covering_overview['#bibs']>=5)].shape

(14989, 6)

In [48]:
acl_ids_for_out = df_covering_overview[(df_covering_overview['%bibs_with_links2bibs']>=0.9) & (df_covering_overview['#bibs']>=5)]

In [49]:
acl_ids_for_out_index = list(acl_ids_for_out.index)

In [50]:
acl_ids_for_out_index_keys = {val:1 for val in acl_ids_for_out_index}

In [51]:
acl_ids_not_for_out = [article['paper_id'] for article in acl_only_articles if article['paper_id'] not in acl_ids_for_out_index_keys]

In [52]:
acl_only_articles = delete_items_from_papers(acl_ids_not_for_out,acl_only_articles)

In [53]:
len(acl_only_articles)

7259

In [54]:
[article['paper_id'] for article in acl_only_articles if article['paper_id'] not in acl_ids_for_out_index_keys]

[]

In [55]:
acl_only_articles[0]

{'paper_id': '10164018',
 'metadata': {'title': 'Reader-Aware Multi-Document Summarization: An Enhanced Model and The First Dataset',
  'authors': [{'first': 'Piji', 'middle': [], 'last': 'Li', 'suffix': ''},
   {'first': 'Lidong', 'middle': [], 'last': 'Bing', 'suffix': ''},
   {'first': 'Wai', 'middle': [], 'last': 'Lam', 'suffix': ''}],
  'abstract': 'We investigate the problem of reader-aware multi-document summarization (RA-MDS) and introduce a new dataset for this problem. To tackle RA-MDS, we extend a variational auto-encodes (VAEs) based MDS framework by jointly considering news documents and reader comments. To conduct evaluation for summarization performance, we prepare a new dataset. We describe the methods for data collection, aspect annotation, and summary writing as well as scrutinizing by experts. Experimental results show that reader comments can improve the summarization performance, which also demonstrates the usefulness of the proposed dataset. The annotated dataset 

In [56]:
article_bibs['10164018']

{'Multi-document summarization by sentence extraction': ['8294822'],
 'Lexpagerank: Prestige in multi-document text summarization': ['10418456'],
 'Manifold-ranking based topic-focused multidocument summarization': ['532313'],
 'A survey of text summarization techniques': ['556431'],
 'Exploiting category-specific information for multidocument summarization': ['6317274'],
 'Salience estimation via variational auto-encoders for multi-document summarization': ['29562039'],
 "Comments-oriented document summarization: Understanding documents with readers' feedback": ['13723748'],
 'Social context summarization': ['704517'],
 'Abstractive multidocument summarization via phrase selection and merging': ['8377315'],
 'Autoencoding variational bayes': [],
 'Stochastic backpropagation and approximate inference in deep generative models': ['16895865']}

In [57]:
article_bibs['14472576']

{'Large-scale semantic parsing via schema matching and lexicon extension': ['2265838'],
 'Scaling semantic parsers with on-the-fly ontology matching': ['14341841'],
 'Semantic parsing on Freebase from question-answer pairs': ['6401679'],
 'Using semantic unification to generate regular expressions from natural language': ['6216733'],
 'A joint model of language and perception for grounded attribute learning': ['2408319'],
 'Understanding natural language commands for robotic navigation and mobile manipulation': ['1078533'],
 'Jointly learning to parse and perceive: Connecting natural language to the physical world': ['10250712'],
 'Learning to map sentences to logical form: Structured classification with probabilistic categorial grammars': ['449252'],
 'Learning synchronous grammars for semantic parsing with lambda calculus': ['9337134'],
 "Driving semantic parsing from the world's response": ['5667590'],
 'Learning dependency-based compositional semantics': ['340852'],
 'Bootstrapping

In [58]:
with open('acl_scenario/acl_only_articles_0_9.json', 'w+') as json_fil:
    json.dump(list(acl_only_articles), json_fil, separators=(',', ':'))
json_fil.close()    

In [59]:
len(acl_only_articles)

7259

In [60]:
acl_paper_ids = {article['paper_id']:1 for article in acl_only_articles }
len(acl_paper_ids)

7259

In [61]:
overview_papers_list = [v for k,v in overview_papers.items() if k in acl_paper_ids]

In [62]:
len(overview_papers_list)

7259

In [63]:
with open('acl_scenario/overview_papers_0_9.json', 'w+') as json_fil:
    json.dump(overview_papers_list, json_fil, separators=(',', ':'))
json_fil.close()    

In [64]:
acl_out_ids = []
for paper in acl_only_articles:
#     print(paper['paper_id'])
    for out in article_bibs[paper['paper_id']].values():
        if len(out)>0:
            acl_out_ids+=out
#             print(out)
#     print(acl_out_ids)
#     print(10*'==')

In [65]:
len(acl_out_ids),len(set(acl_out_ids))

(82636, 26887)

In [66]:
acl_out_ids_keys = {val:1 for val in set(acl_out_ids)}

In [67]:
with open('acl_scenario/acl_out_ids.txt', 'w') as f_in_file:
    for item in set(acl_out_ids):
        f_in_file.write("%s\n" % item)
f_in_file.close()

In [68]:
acl_only_articles[0].keys()

dict_keys(['paper_id', 'metadata', 's2_pdf_hash', 'grobid_parse', 'latex_parse'])

In [69]:
acl_only_articles[0]['grobid_parse'].keys()

dict_keys(['abstract', 'body_text', 'ref_entries', 'bib_entries'])

In [70]:
acl_only_articles[0]['grobid_parse']['bib_entries']

{'BIBREF0': {'ref_id': 'b0',
  'title': 'Neural machine translation by jointly learning to align and translate',
  'authors': [{'first': 'Dzmitry',
    'middle': [],
    'last': 'Bahdanau',
    'suffix': ''},
   {'first': 'Kyunghyun', 'middle': [], 'last': 'Cho', 'suffix': ''},
   {'first': 'Yoshua', 'middle': [], 'last': 'Bengio', 'suffix': ''}],
  'year': 2015,
  'venue': 'ICLR',
  'volume': '',
  'issn': '',
  'pages': '',
  'other_ids': {},
  'links': '11212020'},
 'BIBREF1': {'ref_id': 'b1',
  'title': 'Theano: new features and speed improvements',
  'authors': [{'first': 'Frédéric',
    'middle': [],
    'last': 'Bastien',
    'suffix': ''},
   {'first': 'Pascal', 'middle': [], 'last': 'Lamblin', 'suffix': ''},
   {'first': 'Razvan', 'middle': [], 'last': 'Pascanu', 'suffix': ''},
   {'first': 'James', 'middle': [], 'last': 'Bergstra', 'suffix': ''},
   {'first': 'Ian', 'middle': [], 'last': 'Goodfellow', 'suffix': ''},
   {'first': 'Arnaud', 'middle': [], 'last': 'Bergeron', 'su

### Cоберем данные статьи

### нам для обучения сценария, кроме метаданных ничего не нужно!

In [71]:
dir_name = 'acl_scenario/'

In [72]:
del overview_papers

In [73]:
epoch = -1
all_articles = []
for file in tqdm_notebook([file for file in f_zips if '.gz' in file]):
    txt_input = "../../gorc/"+file
    
    zip_num = file.split('.')[0]
    epoch+=1
    
    with gzip.open(txt_input, "r") as read_file:
#         print(txt_input)
        try:
            json_list = list(read_file)
        except:
#             epoch-=1
            print(f'ERROR epoch={epoch}')
            
    for num, jsonchik in enumerate(json_list):
        try:
            result = json.loads(jsonchik)
    #             context_dict = get_out_citation(result,context_dict)
            if result['paper_id'] in acl_out_ids_keys:
                latex_parse_abstract = None
                latex_parse_bib_entries = None
                
                grobid_parse_abstract = None
                grobid_parse_bib_entries = None
                if result['grobid_parse']:
                    if result['grobid_parse']['abstract']:#and result['grobid_parse']['body_text']
                        grobid_parse_abstract = result['grobid_parse']['abstract']
                    if result['grobid_parse']['bib_entries']:
                        grobid_parse_bib_entries = result['grobid_parse']['bib_entries']
                if result['latex_parse']:
                    if result['latex_parse']['abstract']:# and result['latex_parse']['body_text']:
                        latex_parse_abstract = result['latex_parse']['abstract']
                    if result['latex_parse']['bib_entries']:
                        latex_parse_bib_entries = result['latex_parse']['bib_entries']
                out_paper = {'paper_id': result['paper_id'], 'metadata':result['metadata'],'s2_pdf_hash':result['s2_pdf_hash'],
                             'grobid_parse_abstract':grobid_parse_abstract,'latex_parse_abstract':latex_parse_abstract,
                             'grobid_parse_bib_entries':grobid_parse_bib_entries,'latex_parse_bib_entries':latex_parse_bib_entries
                            }
                all_articles.append(out_paper)
        except:
                print('EXCEPT')
                continue
#     context_dict = get__in_out_papers(all_articles, zip_num, context_dict)
    
    read_file.close()
    if epoch % 2000 == 0:
        name_json = dir_name+'acl_out_new_'+str(epoch)+'.json' 
        with open(name_json, 'w') as json_fil:
            json.dump(all_articles, json_fil)
        json_fil.close()
        
        print('epoch =',epoch,'|',len(all_articles))
        all_articles = []  # main for batches

epoch = 0 | 5
epoch = 2000 | 5327
epoch = 4000 | 5290
epoch = 6000 | 5441
ERROR epoch=7369
epoch = 8000 | 5405
epoch = 10000 | 5419



In [146]:
epoch = -1
all_articles = []
for file in tqdm_notebook([file for file in f_zips if '.gz' in file]):
    txt_input = "../../gorc/"+file
    
    zip_num = file.split('.')[0]
    epoch+=1
    
    with gzip.open(txt_input, "r") as read_file:
#         print(txt_input)
        try:
            json_list = list(read_file)
        except:
#             epoch-=1
            print(f'ERROR epoch={epoch}')
            
    for num, jsonchik in enumerate(json_list):
        try:
            result = json.loads(jsonchik)
    #             context_dict = get_out_citation(result,context_dict)
            if result['paper_id'] in acl_out_ids_keys:
                if result['grobid_parse'] and result['grobid_parse']['body_text']:
                    all_articles.append(result)
                elif result['latex_parse'] and result['latex_parse']['body_text']:
                    all_articles.append(result)
                all_articles.append(out_paper)
        except:
                print('EXCEPT')
                continue
#     context_dict = get__in_out_papers(all_articles, zip_num, context_dict)
    
    read_file.close()
    if epoch % 1000 == 0:
        name_json = dir_name+'acl_entire_papers_out'+str(epoch)+'.json' 
        with open(name_json, 'w') as json_fil:
            json.dump(all_articles, json_fil)
        json_fil.close()
        
        print('epoch =',epoch,'|',len(all_articles))
        all_articles = []  # main for batches

epoch = 0 | 7
epoch = 1000 | 4305
epoch = 2000 | 4248
epoch = 3000 | 4300
epoch = 4000 | 4258
epoch = 5000 | 4427
epoch = 6000 | 4312
epoch = 7000 | 4326
ERROR epoch=7369
epoch = 8000 | 4406
epoch = 9000 | 4321
epoch = 10000 | 4445



ещё

In [74]:
del all_articles
del result
del acl_only_articles

#### Не запускал

In [75]:
with open("acl_only_json_list_10000.json", "r") as read_file:
    acl_only_articles = json.load(read_file)
print(len(acl_only_articles))
read_file.close()

41660


In [76]:
acl_ids_for_out = df_covering_overview[(df_covering_overview['%bibs_with_links2bibs']>=0.8) & (df_covering_overview['#bibs']>=5)]
acl_ids_for_out_index = list(acl_ids_for_out.index)
acl_ids_for_out_index_keys = {val:1 for val in acl_ids_for_out_index}
acl_ids_not_for_out = [article['paper_id'] for article in acl_only_articles if article['paper_id'] not in acl_ids_for_out_index_keys]

In [77]:
len(acl_ids_for_out),len(acl_ids_not_for_out),len(acl_only_articles)

(14989, 26671, 41660)

In [78]:
acl_only_articles = delete_items_from_papers(acl_ids_not_for_out,acl_only_articles)

In [79]:
len(acl_only_articles)

14989

In [80]:
acl_out_ids = []
for paper in acl_only_articles:
#     print(paper['paper_id'])
    for out in article_bibs[paper['paper_id']].values():
        if len(out)>0:
            acl_out_ids+=out
#             print(out)
#     print(acl_out_ids)
#     print(10*'==')

In [81]:
len(acl_out_ids),len(set(acl_out_ids))

(158049, 42748)

In [82]:
with open('acl_scenario/acl_out_ids_0_8.txt', 'w') as f_in_file:
    for item in set(acl_out_ids):
        f_in_file.write("%s\n" % item)
f_in_file.close()

In [66]:
with open('acl_scenario/acl_only_articles_0_8.json', 'w+') as json_fil:
    json.dump(list(acl_only_articles), json_fil, separators=(',', ':'))
json_fil.close()    

In [83]:
acl_out_ids_keys = {val:1 for val in acl_out_ids}

In [84]:
len(acl_out_ids_keys)

42748

In [85]:
epoch = -1
all_articles = []
for file in tqdm_notebook([file for file in f_zips if '.gz' in file]):
    txt_input = "../../gorc/"+file
    
    zip_num = file.split('.')[0]
    epoch+=1
    
    with gzip.open(txt_input, "r") as read_file:
#         print(txt_input)
        try:
            json_list = list(read_file)
        except:
#             epoch-=1
            print(f'ERROR epoch={epoch}')
            
    for num, jsonchik in enumerate(json_list):
        try:
            result = json.loads(jsonchik)
    #             context_dict = get_out_citation(result,context_dict)
            if result['paper_id'] in acl_out_ids_keys:
                latex_parse_abstract = None
                latex_parse_bib_entries = None
                
                grobid_parse_abstract = None
                grobid_parse_bib_entries = None
                if result['grobid_parse']:
                    if result['grobid_parse']['abstract']:#and result['grobid_parse']['body_text']
                        grobid_parse_abstract = result['grobid_parse']['abstract']
                    if result['grobid_parse']['bib_entries']:
                        grobid_parse_bib_entries = result['grobid_parse']['bib_entries']
                if result['latex_parse']:
                    if result['latex_parse']['abstract']:# and result['latex_parse']['body_text']:
                        latex_parse_abstract = result['latex_parse']['abstract']
                    if result['latex_parse']['bib_entries']:
                        latex_parse_bib_entries = result['latex_parse']['bib_entries']
                out_paper = {'paper_id': result['paper_id'], 'metadata':result['metadata'],'s2_pdf_hash':result['s2_pdf_hash'],
                             'grobid_parse_abstract':grobid_parse_abstract,'latex_parse_abstract':latex_parse_abstract,
                             'grobid_parse_bib_entries':grobid_parse_bib_entries,'latex_parse_bib_entries':latex_parse_bib_entries
                            }
                all_articles.append(out_paper)
        except:
                print('EXCEPT')
                continue
#     context_dict = get__in_out_papers(all_articles, zip_num, context_dict)
    
    read_file.close()
    if epoch % 2000 == 0:
        name_json = dir_name+'acl_out_0_8_'+str(epoch)+'.json' 
        with open(name_json, 'w') as json_fil:
            json.dump(all_articles, json_fil)
        json_fil.close()
        
        print('epoch =',epoch,'|',len(all_articles))
        all_articles = []  # main for batches

epoch = 0 | 7

